In [ ]:
# HIDDEN
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
def r_scatter(r):
    """Generate a scatter plot with a correlation approximately r"""
    # You don't have to understand how this function works.
    plots.figure(figsize=(5,5))
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.scatter(x, y, color='darkblue', s=20)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)

## Prediction and association

In [ ]:
hybrid = Table.read_table('hybrid.csv')
hybrid

In [ ]:
hybrid.hist('msrp')

In [ ]:
hybrid.sort('msrp', descending=True)

In [ ]:
hybrid.scatter('mpg', 'msrp')

In [ ]:
hybrid.scatter('acceleration', 'msrp')

In [ ]:
suv = hybrid.where('class', 'SUV')
suv

In [ ]:
suv.hist('msrp')

In [ ]:
suv.scatter('acceleration', 'msrp')

In [ ]:
suv.scatter('mpg', 'msrp')

In [ ]:
suv.scatter('mpg', 'msrp')

In [ ]:
galton = Table.read_table('galton.csv')
galton

In [ ]:
galton.scatter('father', 'mother')

In [ ]:
galton.scatter('father', 'midparentHeight')

In [ ]:
heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
)
heights

In [ ]:
heights.scatter('MidParent')

In [ ]:
def predict_child(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.5 inches.
    """
    
    close_points = heights.where('MidParent', are.between(h-0.5, h + 0.5))
    return close_points.column('Child').mean()   

In [ ]:
heights_with_predictions = heights.with_column(
    'Prediction', heights.apply(predict_child, 'MidParent')
)

In [ ]:
heights_with_predictions.scatter('MidParent')

## Association ##

In [ ]:
def standard_units(x):
    "Convert any array of numbers to standard units."
    return (x - np.average(x)) / np.std(x)

In [ ]:
suv.scatter('mpg', 'msrp')

In [ ]:
Table().with_columns(
    'mpg (standard units)',  standard_units(suv.column('mpg')), 
    'msrp (standard units)', standard_units(suv.column('msrp'))
).scatter(0, 1)
plots.xlim(-3, 3)
plots.ylim(-3, 3);

In [ ]:
Table().with_columns(
    'acceleration (standard units)', standard_units(suv.column('acceleration')), 
    'msrp (standard units)',         standard_units(suv.column('msrp'))
).scatter(0, 1)
plots.xlim(-3, 3)
plots.ylim(-3, 3);

## Calculating $r$ ##

In [ ]:
x = np.arange(1, 7, 1)
y = make_array(2, 3, 1, 5, 2, 7)
t = Table().with_columns(
        'x', x,
        'y', y
    )
t

In [ ]:
t.scatter('x', 'y', s=50, color='red')

In [ ]:
x_su = standard_units(x)
y_su = standard_units(y)

In [ ]:
t = t.with_columns(
        'x (standard units)', x_su,
        'y (standard units)', y_su
    )
t

In [ ]:
t.scatter(2, 3, s=50, color='red')

In [ ]:
su_product = x_su * y_su

In [ ]:
t = t.with_column('product of standard units', su_product)
t

In [ ]:
# r is the average of the products of the standard units

r = np.average(su_product)
r

In [ ]:
def correlation(t, x, y):
    """t is a table; x and y are column labels"""
    x_in_standard_units = standard_units(t.column(x))
    y_in_standard_units = standard_units(t.column(y))
    return np.average(x_in_standard_units * y_in_standard_units)

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
correlation(suv, 'mpg', 'msrp')

In [ ]:
correlation(suv, 'acceleration', 'msrp')

## Correlation ##

In [ ]:
r_scatter(-1)

In [ ]:
r_scatter(0.5)

In [ ]:
r_scatter(-0.5)

In [ ]:
r_scatter(0)

### Switching Axes ###

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
t.scatter('x', 'y', s=50, color='red')

In [ ]:
t.scatter('y', 'x', s=50, color='red')

In [ ]:
correlation(t, 'y', 'x')

### Nonlinearity ###

In [ ]:
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=50, color='r')

In [ ]:
correlation(nonlinear, 'x', 'y')

### Outliers ###

In [ ]:
line = Table().with_columns(
        'x', make_array(1, 2, 3, 4),
        'y', make_array(1, 2, 3, 4)
    )
line.scatter('x', 'y', s=50, color='r')

In [ ]:
correlation(line, 'x', 'y')

In [ ]:
outlier = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, 3, 4, 0)
    )
outlier.scatter('x', 'y', s=50, color='r')

In [ ]:
correlation(outlier, 'x', 'y')

In [ ]:
outlier2 = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, -2, 4, 5)
    )
outlier2.scatter('x', 'y', s=50, color='r')

In [ ]:
correlation(outlier2, 'x', 'y')

### Ecological correlations

In [ ]:
election16 = Table.read_table('election2016_state.csv')
election16

In [ ]:
election16.scatter('Median Income', 'Percent voting for Clinton')

In [ ]:
party = Table.read_table('party_affiliation2016.csv')
party

In [ ]:
party.exclude(6).barh('Income')

In [ ]:
sat2014 = Table.read_table('sat2014.csv').sort('State')
sat2014.show(4)

In [ ]:
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
correlation(sat2014, 'Critical Reading', 'Math')